### Data Aggregation and Group Operation

Categorizing a data set and applying a fucntion to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow.

After loading, merging, and preparing a data set, a familiar task is to compute group statistics or possibly `pivot tables` for reporting or visualization purpose. 

In this chapter, you will lean how to:


 - Split a pandas object into pieces using one or more keys(in the form of functions array, or DataFrame columns names)
 
 
 - Computing group summary statistic, like `count`, `mean`, `standard deviation`, or `a user-defind function`.


 - Apply a varying set of functions to each column of DataFrame.
 
 
 - Apply within-group transformations or other manipulations, like normalization linear regression, rank, or subset selection.
 
 
 - Compute pivot tables and cross-tabulations.
 
 
 - Perform quantile analysis and other data-derived group analyses.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

***Group operations can be separated by three steps, `split`, `apply` and `combine`.***

In the first stage of the process, data contained in pandas object, whether a Series, DataFrame, or otherwise, is `split` into groups based on one or more `keys` that you provide.

The splitting is performed on a particular axis of an object. A DataFrame can be grouped on its `rows(axis = 0)` or its `columns(axis = 1).`

***Once this is done, a function `applied` to each group, producing a new value.***

***Finally, the results of all those function applications are `combined` into a result object.***

Each grouping key can **take many forms**, and the keys **don't have to be all of the same type**:

 - A list or array of values that is the same length as the axis being grouped.
 
 
 - A value indicating a column name in a DataFrame.
 
 
 - A dict or Series giving a correspondence between the values on the axis being grouped and the group names.
 
 
 - A function to be invoked on the axis index or the individual labels in the index.

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.936546,-1.046599,a,one
1,1.140404,1.058859,a,two
2,0.007245,-1.332424,b,one
3,-0.059206,-0.638958,b,two
4,1.278121,0.042200,a,one


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a    0.160660
b   -0.025981
Name: data1, dtype: float64

In [4]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.329212
      two     1.140404
b     one     0.007245
      two    -0.059206
Name: data1, dtype: float64

In [5]:
means.unstack(level = 1) #從內到外

key2,one,two
key1,,
a,-0.329212,1.140404
b,0.007245,-0.059206


In these examples, the group keys are all Series, though they could be any arrays of the right length.

In [6]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', "Ohio"])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby(by = [states, years]).mean()

California  2005    1.140404
            2006    0.007245
Ohio        2005   -0.997876
            2006    1.278121
Name: data1, dtype: float64

***Frequently the grouping information to be found in the same DataFrame as the data you want to work on.***

You may have noticed in the first case `df.groupby('key1').mean()` that there is no `key2` column in the result. 

Because `df["key2"]` is not numeric data, it is said to be a `nuisance column`, which is therefore excluded from the result.

In [7]:
print(df.groupby('key1').mean())
print(df.groupby(['key1', 'key2']).mean())

         data1     data2
key1                    
a     0.160660  0.018153
b    -0.025981 -0.985691
              data1     data2
key1 key2                    
a    one  -0.329212 -0.502200
     two   1.140404  1.058859
b    one   0.007245 -1.332424
     two  -0.059206 -0.638958


***Regardless of the objective in using `groupby`, a generally useful GroupBy method is `size` which return a Series containing group sizes.***

***As of this writting, any missing values in a group key will be `excluded` from the result.***

***It's possible(and, in fact, quite likely), that by the time you are reading this there will be an option to include the `NA` group in the result.***

In [8]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

The GroupBy object supports iteration, generating a seqence of `2-tuples` containing the ***group name along with the chunk of data***.

In [9]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -1.936546 -1.046599    a  one
1  1.140404  1.058859    a  two
4  1.278121  0.042200    a  one
b
      data1     data2 key1 key2
2  0.007245 -1.332424    b  one
3 -0.059206 -0.638958    b  two


In the case of multiple keys, the first element in the tuple will be a tuple of key values.

In [10]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
      data1     data2 key1 key2
0 -1.936546 -1.046599    a  one
4  1.278121  0.042200    a  one
a two
      data1     data2 key1 key2
1  1.140404  1.058859    a  two
b one
      data1     data2 key1 key2
2  0.007245 -1.332424    b  one
b two
      data1     data2 key1 key2
3 -0.059206 -0.638958    b  two


Of course, you can choose to do whatever you want with the pieces of data.

A recipe you may find useful is computing a dict of the data pieces as a one-liner.

In [11]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,0.007245,-1.332424,b,one
3,-0.059206,-0.638958,b,two


In [12]:
list(df.groupby('key1'))

[('a',       data1     data2 key1 key2
  0 -1.936546 -1.046599    a  one
  1  1.140404  1.058859    a  two
  4  1.278121  0.042200    a  one), ('b',       data1     data2 key1 key2
  2  0.007245 -1.332424    b  one
  3 -0.059206 -0.638958    b  two)]

In [13]:
temp = dict([['a', [1, 2, 3]], ['c', [4, 5, 6, 7]]])
print(temp['a'])
print(temp['c'])

[1, 2, 3]
[4, 5, 6, 7]


In [14]:
dict(list([tuple(["a", [3, 5, 6]]), tuple(["b", [3, 5, 6]])]))

{'a': [3, 5, 6], 'b': [3, 5, 6]}

***By default `groupby` groups on `axis = 0`, but you can group on any of the other axes.*** 

We would group the columns of our example df here by `dtype`.

In [15]:
grouped = df.groupby(by = df.dtypes, axis = 1)
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -1.936546 -1.046599
 1  1.140404  1.058859
 2  0.007245 -1.332424
 3 -0.059206 -0.638958
 4  1.278121  0.042200, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of `selecting those columns` for aggregation.

Especially for large data sets, it may be describe to aggregate only a few columns. 

For example, in the above data set, to compute means for just `data2` column and get the result as a DataFrame.

In [16]:
for name, group in df.groupby('key1')[['data1']]:
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -1.936546 -1.046599    a  one
1  1.140404  1.058859    a  two
4  1.278121  0.042200    a  one
b
      data1     data2 key1 key2
2  0.007245 -1.332424    b  one
3 -0.059206 -0.638958    b  two


In [17]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one    -0.502200
      two     1.058859
b     one    -1.332424
      two    -0.638958
Name: data2, dtype: float64

In [18]:
dict(list(df.groupby(['key1', 'key2'])['data2'])) # a piece of data

{('a', 'one'): 0   -1.046599
 4    0.042200
 Name: data2, dtype: float64, ('a', 'two'): 1    1.058859
 Name: data2, dtype: float64, ('b', 'one'): 2   -1.332424
 Name: data2, dtype: float64, ('b', 'two'): 3   -0.638958
 Name: data2, dtype: float64}

### Grouping with Dicts and Series

Grouping information many exiset in a form other an array.

In [19]:
people = pd.DataFrame(np.random.randn(6, 6), 
                      columns = ['a', 'b', 'c', 'd', 'e', 'f'], 
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis', 'Tom'])

people

,a,b,c,d,e,f
Joe,1.562089,-0.727263,2.452854,-1.448111,1.009144,-0.746483
Steve,-0.626949,0.047157,0.195637,2.070124,-0.747004,-0.704328
Wes,-1.799041,1.568818,-0.046071,-0.090005,0.305684,-0.023407
Jim,-1.415039,0.257306,1.128847,0.820776,0.847937,2.770388
Travis,-0.657080,0.185517,1.402789,-0.097816,2.525185,0.744277
Tom,2.269964,-1.464651,-0.137389,-0.209731,1.779395,1.705856


In [20]:
people.loc[2:3, ['b', 'c']] = np.nan # Add a few NA values

people

,a,b,c,d,e,f
Joe,1.562089,-0.727263,2.452854,-1.448111,1.009144,-0.746483
Steve,-0.626949,0.047157,0.195637,2.070124,-0.747004,-0.704328
Wes,-1.799041,NaN,NaN,-0.090005,0.305684,-0.023407
Jim,-1.415039,0.257306,1.128847,0.820776,0.847937,2.770388
Travis,-0.657080,0.185517,1.402789,-0.097816,2.525185,0.744277
Tom,2.269964,-1.464651,-0.137389,-0.209731,1.779395,1.705856


Now, suppose I have a group correspondence for the columns and want to sum together the columns by group.

You could easily construct an array from this dict to pass to `groupby`, but instead we can just pass the dict.

In [21]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [22]:
by_column = people.groupby(mapping, axis = 1)
by_column.sum()

,blue,orange,red
Joe,1.004742,-0.746483,1.843971
Steve,2.265761,-0.704328,-1.326796
Wes,-0.090005,-0.023407,-1.493357
Jim,1.949623,2.770388,-0.309796
Travis,1.304974,0.744277,2.053622
Tom,-0.347120,1.705856,2.584709


The same functionality holds for Series, which can be viewed as a fixed size mapping.

When I used Series as group keys in the above examples, pandas does, in fact, inpect each Series to ensure that its index is aligned with the axis it's grouping.

In [23]:
map_series = pd.Series(data = mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series, axis = 1).count()

,blue,orange,red
Joe,2,1,3
Steve,2,1,3
Wes,1,1,2
Jim,2,1,3
Travis,2,1,3
Tom,2,1,3


### Grouping with Functions

Using Python functions in what can be fairly creative ways is a more abstract way of defining a group mapping compared with a dict or Series.

***Any function passed as a group key will be called once per index, with return values being used as the group names.***

More concretely, consider the example DataFrame from the pervious section, which has people's name as index values.

***Suppose you wanted to group by the length of names, you could compute an array of string lengths, but instead you can just pass the `len` function.***

In [25]:
print(people)
print("-"*50)
people.groupby(len).sum()

               a         b         c         d         e         f
Joe     1.562089 -0.727263  2.452854 -1.448111  1.009144 -0.746483
Steve  -0.626949  0.047157  0.195637  2.070124 -0.747004 -0.704328
Wes    -1.799041       NaN       NaN -0.090005  0.305684 -0.023407
Jim    -1.415039  0.257306  1.128847  0.820776  0.847937  2.770388
Travis -0.657080  0.185517  1.402789 -0.097816  2.525185  0.744277
Tom     2.269964 -1.464651 -0.137389 -0.209731  1.779395  1.705856
--------------------------------------------------


,a,b,c,d,e,f
3,0.617974,-1.934607,3.444312,-0.927072,3.942161,3.706354
5,-0.626949,0.047157,0.195637,2.070124,-0.747004,-0.704328
6,-0.657080,0.185517,1.402789,-0.097816,2.525185,0.744277


***Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally.***

In [26]:
key_list = ['one', 'one', 'one', 'two', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e         f
3 one -1.799041 -0.727263  2.452854 -1.448111  0.305684 -0.746483
  two -1.415039 -1.464651 -0.137389 -0.209731  0.847937  1.705856
5 one -0.626949  0.047157  0.195637  2.070124 -0.747004 -0.704328
6 two -0.657080  0.185517  1.402789 -0.097816  2.525185  0.744277

### Grouping by Index Levels

A final convenience for hierarchically-indexed data sets is the ability to aggregate using one of the levels of an axis index.

To do this, pass the level number or name using the `level` keyword.

In [27]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"], 
                                     [1, 3, 5, 1, 3]], names = ['cty', 'tensor'])

In [28]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns = columns)
hier_df

cty           US                            JP          
tensor         1         3         5         1         3
0       0.074987 -0.311536 -0.856349  0.150448  0.517238
1       0.908726 -0.288798 -0.646600 -1.153661 -0.145016
2       0.114413 -0.418209 -0.625865 -0.500055  0.070979
3       1.534744 -1.653870 -0.204162  0.678256 -0.325460

In [29]:
hier_df.columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tensor'])

In [30]:
hier_df.groupby(level = 'cty', axis = 1).sum() # groupby by columns 

cty,JP,US
0,0.667686,-1.092898
1,-1.298677,-0.026671
2,-0.429075,-0.929661
3,0.352796,-0.323288


### Data Aggregation

By aggregation, I am generally referring to any data transformation that produces scalar values from array.

***You can use `aggregations` of your own devising and additionally call any method that is also defind on the grouped object.***

While `quantile` is not explicity implemented for `Groupby`, it is a Series method and thus available for use.

***Internally, `Groupby` efficiently slices up the Serices, calls `piece.qunatile(0.9)` for each piece, then assembles those result together into the result object.***

In [31]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.250578
b    0.000600
Name: data1, dtype: float64

***To use your own aggregation functions, pass any function that aggregates an array to the `aggregate` or `agg` method.***

Note that `quantile` performs linear interploation if there is no value at exactly the passed percentile.

In [32]:
def peak_to_peak(arr):
    return(arr.max() - arr.min())

In [33]:
grouped.aggregate(peak_to_peak)

,data1,data2
key1,,
a,3.214667,2.105458
b,0.066452,0.693466


In [34]:
tips = pd.read_csv(".\\pydata-book\\examples\\tips.csv")
tips.head(3)

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3


In [35]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

In [36]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


### Column-wise and Multiple Function Application

Aggregating a Series or all of the columns of a DataFrame is a matter of using `aggregate` with the desired function or calling a method like `mean` or `std`.

***You may want to aggregate using a different function depending on the column or multiple functions at once.***

***If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions.***

In [37]:
grouped = tips.groupby(['time', 'smoker'])
grouped['tip_pct'].agg('mean')

time    smoker
Dinner  No        0.158653
        Yes       0.160828
Lunch   No        0.160920
        Yes       0.170404
Name: tip_pct, dtype: float64

In [38]:
grouped['tip_pct'].agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
time   smoker                                  
Dinner No      0.158653  0.040458      0.235193
       Yes     0.160828  0.095153      0.674707
Lunch  No      0.160920  0.038989      0.193350
       Yes     0.170404  0.042770      0.169300

You don't need to accept the names that GroupBy gives to the columns.

Notable `lambda` functions have the same `<lambda>` which make them hard to identify (you can see for yourself by looking at function's `__name__` attribute).

***As such, if you pass a list of `(name, function)` tuples, the first element of each tuple will be used as the Dataframe column names (you can think of a list of 2-tuples as an ordered mapping).***

In [39]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
time   smoker                    
Dinner No      0.158653  0.040458
       Yes     0.160828  0.095153
Lunch  No      0.160920  0.038989
       Yes     0.170404  0.042770

***With DataFrame, you have more options as you can specify a list if functions to apply all of the columns or different functions per column.***

Suppose we wanted to compute the same three statistics for the `tip_pct` and `total_bill` columns.

***As you can see, the resulting DataFrame has hierarchical columns, the same as you would get aggregating each column separately and using `concat` to glue the results together using the column names as the `key` argument.***

In [40]:
functions = ['count', 'mean', 'max']

In [41]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
time   smoker                                                         
Dinner No         106  0.158653  0.291990        106  20.095660  48.33
       Yes         70  0.160828  0.710345         70  21.859429  50.81
Lunch  No          45  0.160920  0.266312         45  17.050889  41.19
       Yes         23  0.170404  0.259314         23  17.399130  43.11

The same as blow manipulation.

In [42]:
column_names = pd.MultiIndex.from_arrays([['tip_pct', 'tip_pct', 'tip_pct', 'total_bill', 'total_bill', 'total_bill'], 
                                          ['count', 'mean', 'max', 'count', 'mean', 'max']])

In [43]:
res1 = grouped['tip_pct'].agg(functions)
res2 = grouped['total_bill'].agg(functions)

In [44]:
res = pd.concat([res1,res2], axis = 1)
res.columns = column_names
res

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
time   smoker                                                         
Dinner No         106  0.158653  0.291990        106  20.095660  48.33
       Yes         70  0.160828  0.710345         70  21.859429  50.81
Lunch  No          45  0.160920  0.266312         45  17.050889  41.19
       Yes         23  0.170404  0.259314         23  17.399130  43.11

As above, a list of tuples with custom names can be passed.

***Now suppose you wanted to apply potentially different functions to one or more of the columns. ***

***The trick is to pass `dict` to `agg` that contains a mapping of column names to any of the function specifications listed so far.***

In [45]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [46]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
              Durchschnitt Abweichung Durchschnitt  Abweichung
time   smoker                                                 
Dinner No         0.158653   0.001637    20.095660   69.604821
       Yes        0.160828   0.009054    21.859429  104.148753
Lunch  No         0.160920   0.001520    17.050889   59.587154
       Yes        0.170404   0.001829    17.399130   61.958436

In [47]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
time   smoker            
Dinner No       9.0   290
       Yes     10.0   173
Lunch  No       6.7   113
       Yes      5.0    51

In [48]:
grouped.agg({'tip_pct':['min', 'max', 'mean', 'std'], 
             'size': 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
time   smoker                                             
Dinner No      0.056797  0.291990  0.158653  0.040458  290
       Yes     0.035638  0.710345  0.160828  0.095153  173
Lunch  No      0.072961  0.266312  0.160920  0.038989  113
       Yes     0.090014  0.259314  0.170404  0.042770   51

### Returning Aggregated Data in `unindexed` Form

In all of the example up until now, the aggregated data `comes back with an index`, potentially hierarchical composed from the unique group key combinations observed.

***Of course, it's always possible to obtain the result in this format by calling `reset_index` on the result.***

In [49]:
tips.groupby(['time', 'smoker'], as_index = False).mean()

,time,smoker,total_bill,tip,size,tip_pct
0,Dinner,No,20.095660,3.126887,2.735849,0.158653
1,Dinner,Yes,21.859429,3.066000,2.471429,0.160828
2,Lunch,No,17.050889,2.673778,2.511111,0.160920
3,Lunch,Yes,17.399130,2.834348,2.217391,0.170404


In [50]:
tips.groupby(['time', 'smoker']).mean().reset_index()

,time,smoker,total_bill,tip,size,tip_pct
0,Dinner,No,20.095660,3.126887,2.735849,0.158653
1,Dinner,Yes,21.859429,3.066000,2.471429,0.160828
2,Lunch,No,17.050889,2.673778,2.511111,0.160920
3,Lunch,Yes,17.399130,2.834348,2.217391,0.170404


In [51]:
temp = tips.groupby(['time', 'smoker']).mean().reset_index()

In [52]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], 
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

### Group-wise Operations and Transforamtions

Aggregation is only one kind of group operation. It is a special case in the more general class of data transformations.

***The methods of `transform` and `apply` will enable you to do many other kinds of group operations.***

Suppose, instead, we wanted to add a column to a DataFrame containing group means for each index. One way to do this is to aggregate, then merge.

In [53]:
df

,data1,data2,key1,key2
0,-1.936546,-1.046599,a,one
1,1.140404,1.058859,a,two
2,0.007245,-1.332424,b,one
3,-0.059206,-0.638958,b,two
4,1.278121,0.042200,a,one


In [54]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,0.160660,0.018153
b,-0.025981,-0.985691


In [55]:
# merge on : Left_on = 'key1', right data use index as a combination key.
pd.merge(df, k1_means, left_on = 'key1', right_index = True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,-1.936546,-1.046599,a,one,0.160660,0.018153
1,1.140404,1.058859,a,two,0.160660,0.018153
4,1.278121,0.042200,a,one,0.160660,0.018153
2,0.007245,-1.332424,b,one,-0.025981,-0.985691
3,-0.059206,-0.638958,b,two,-0.025981,-0.985691


In [56]:
## the same as the blow method
k1_means = df.groupby('key1').mean().add_prefix('mean_').reset_index()
k1_means

,key1,mean_data1,mean_data2
0,a,0.160660,0.018153
1,b,-0.025981,-0.985691


In [57]:
pd.merge(df, k1_means, on = 'key1')

,data1,data2,key1,key2,mean_data1,mean_data2
0,-1.936546,-1.046599,a,one,0.160660,0.018153
1,1.140404,1.058859,a,two,0.160660,0.018153
2,1.278121,0.042200,a,one,0.160660,0.018153
3,0.007245,-1.332424,b,one,-0.025981,-0.985691
4,-0.059206,-0.638958,b,two,-0.025981,-0.985691


This works, but is somewhat inflexible. You can think the operation as transforming the two data columns using the `np.mean` function.

In [58]:
people = pd.DataFrame(np.random.randn(6, 6), 
                      columns = ['a', 'b', 'c', 'd', 'e', 'f'], 
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis', 'Tom'])

people

,a,b,c,d,e,f
Joe,0.223576,-0.168816,-0.699496,1.953425,-0.765692,-1.350505
Steve,0.675597,-1.991750,0.350548,0.300710,-0.284906,0.319256
Wes,0.038540,-0.491659,-0.015667,0.530562,0.188289,-0.348748
Jim,0.526856,-0.426417,0.126912,-1.269683,-0.179284,-0.274010
Travis,0.230544,1.763303,-0.375602,-0.318032,-0.279813,-1.535779
Tom,2.120142,-1.445107,0.087408,0.020164,0.090707,0.022898


In [59]:
people.loc[2:3, ['b', 'c']] = np.nan # Add a few NA values

people

,a,b,c,d,e,f
Joe,0.223576,-0.168816,-0.699496,1.953425,-0.765692,-1.350505
Steve,0.675597,-1.991750,0.350548,0.300710,-0.284906,0.319256
Wes,0.038540,NaN,NaN,0.530562,0.188289,-0.348748
Jim,0.526856,-0.426417,0.126912,-1.269683,-0.179284,-0.274010
Travis,0.230544,1.763303,-0.375602,-0.318032,-0.279813,-1.535779
Tom,2.120142,-1.445107,0.087408,0.020164,0.090707,0.022898


In [60]:
people.index = pd.MultiIndex.from_arrays([['one', 'two', 'one', 'two', 'one', 'two'], 
                                          people.index])
people

,,a,b,c,d,e,f
one,Joe,0.223576,-0.168816,-0.699496,1.953425,-0.765692,-1.350505
two,Steve,0.675597,-1.991750,0.350548,0.300710,-0.284906,0.319256
one,Wes,0.038540,NaN,NaN,0.530562,0.188289,-0.348748
two,Jim,0.526856,-0.426417,0.126912,-1.269683,-0.179284,-0.274010
one,Travis,0.230544,1.763303,-0.375602,-0.318032,-0.279813,-1.535779
two,Tom,2.120142,-1.445107,0.087408,0.020164,0.090707,0.022898


In [61]:
people1 = people.reset_index()
people1.index = people1['level_1']
people1 = people1.drop(columns = ['level_1'])
people1

,level_0,a,b,c,d,e,f
level_1,,,,,,,
Joe,one,0.223576,-0.168816,-0.699496,1.953425,-0.765692,-1.350505
Steve,two,0.675597,-1.991750,0.350548,0.300710,-0.284906,0.319256
Wes,one,0.038540,NaN,NaN,0.530562,0.188289,-0.348748
Jim,two,0.526856,-0.426417,0.126912,-1.269683,-0.179284,-0.274010
Travis,one,0.230544,1.763303,-0.375602,-0.318032,-0.279813,-1.535779
Tom,two,2.120142,-1.445107,0.087408,0.020164,0.090707,0.022898


In [62]:
people1.index.name = None
people1 = people1.rename({'level_0': 'class'}, axis = "columns")
people1

,class,a,b,c,d,e,f
Joe,one,0.223576,-0.168816,-0.699496,1.953425,-0.765692,-1.350505
Steve,two,0.675597,-1.991750,0.350548,0.300710,-0.284906,0.319256
Wes,one,0.038540,NaN,NaN,0.530562,0.188289,-0.348748
Jim,two,0.526856,-0.426417,0.126912,-1.269683,-0.179284,-0.274010
Travis,one,0.230544,1.763303,-0.375602,-0.318032,-0.279813,-1.535779
Tom,two,2.120142,-1.445107,0.087408,0.020164,0.090707,0.022898


### Transfrom

***`Transform` applied a finction to each group, then places the results in the appropriate locations.***

***If each group produce a scalar value, it will be propagated.(broadcasted)***

Suppose instead you wanted to subtract the mean value from each group. To do this, creat a demeaning function and pass it to `transform`.

In [63]:
people1.groupby('class').transform(np.mean)

,a,b,c,d,e,f
Joe,0.164220,0.797244,-0.537549,0.721985,-0.285739,-1.078344
Steve,1.107532,-1.287758,0.188289,-0.316270,-0.124494,0.022714
Wes,0.164220,0.797244,-0.537549,0.721985,-0.285739,-1.078344
Jim,1.107532,-1.287758,0.188289,-0.316270,-0.124494,0.022714
Travis,0.164220,0.797244,-0.537549,0.721985,-0.285739,-1.078344
Tom,1.107532,-1.287758,0.188289,-0.316270,-0.124494,0.022714


In [64]:
def demean(arr):
    return arr - arr.mean()

In [65]:
people1.groupby('class').transform(demean)

,a,b,c,d,e,f
Joe,0.059356,-0.966059,-0.161947,1.231440,-0.479953,-0.272161
Steve,-0.431934,-0.703992,0.162259,0.616979,-0.160412,0.296541
Wes,-0.125680,NaN,NaN,-0.191423,0.474028,0.729596
Jim,-0.580676,0.861341,-0.061378,-0.953413,-0.054790,-0.296725
Travis,0.066323,0.966059,0.161947,-1.040017,0.005926,-0.457435
Tom,1.012610,-0.157349,-0.100881,0.336434,0.215201,0.000183


### Apply: General split-apply-combine

Like `aggregate`, `transform` is a more specialized function having rigid requirements.

The passed function must either produce a scalar value to be brocasted(like `np.mean`) or a transformed array of the same size.

***The most general purpose `Groupby` method is `apply`, which is the subject of the rest of this section.***

***`Apply` splits the object being manipulated into pieces, invokes the pass function on each piece, then attempts to concatenate the pieces together.***

In [66]:
tips = pd.read_csv(".\\pydata-book\\examples\\tips.csv")
tips[:5]

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [67]:
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips[:5]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [68]:
def top(df, n = 5, column = 'tip_pct'):
    return df.sort_values(by = column, ascending = False)[:n]

In [69]:
top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


***The `top` function is called on each piece of the DataFrame, then the results are glued together using `pandas.concat`, labeling the pieces with the group names.***

The result therefore has a hierarchical index whose inner level contains index values from the original DataFrame.

***If you pass a function to `apply` that takes other arguments or keywords, you can pass these after the function.***

In [70]:
tips.groupby(by = 'smoker').apply(top, n = 3)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733

In [71]:
tips.groupby(by = ['smoker', 'day']).apply(top, n = 1, column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

### Suppressing the group keys

In the examples above, you see that the resulting object has a hierarchical index formed from the group keys along with the indexs of each piece of the original object.

This can be disabled by `group_keys = False` to `groupby`. 

In [72]:
print(tips.groupby('smoker').apply(top))
print("-"*50)

print(tips.groupby('smoker', group_keys = False).apply(top))
print("-"*50)

print(tips.groupby('smoker', as_index = False).apply(top))

            total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
--------------------------------------------------
     total_bill   tip smoker   day    time  size   tip_pct
232       11.61  3.39     No   Sat  Dinner     2  0.291990
149        7.51  2.00     No  Thur   Lu

In [73]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

### Quantile and Bucket Analysis

Pandas has some tools, in particular `cut` and `qcut` for slicing data up into buckets with bins of your choosing or by sample quantiles.

***Combining these functions with `groupby`, it becomes very simple to perform bucket or quantile analysis on a data set.***

In [74]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 
                      'data2': np.random.randn(1000)})

In [75]:
factor = pd.cut(frame['data2'], 4)
factor[:10]

0     (-0.119, 1.482]
1     (-0.119, 1.482]
2    (-1.721, -0.119]
3     (-0.119, 1.482]
4     (-0.119, 1.482]
5    (-1.721, -0.119]
6      (1.482, 3.084]
7     (-0.119, 1.482]
8    (-1.721, -0.119]
9    (-1.721, -0.119]
Name: data2, dtype: category
Categories (4, interval[float64]): [(-3.329, -1.721] < (-1.721, -0.119] < (-0.119, 1.482] < (1.482, 3.084]]

In [76]:
# adding the level
pd.cut(x = frame['data1'], bins = 4, labels = ['a', 'b', 'c', 'd'])[:3]

0    b
1    d
2    c
Name: data1, dtype: category
Categories (4, object): [a < b < c < d]

***The `Factor` object returned by `cut` can be passed directly to `groupby`.***

In [77]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [78]:
grouped = frame['data2'].groupby(factor)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data2,,,,
"(-3.329, -1.721]",46.0,-1.721495,-2.110130,-3.322435
"(-1.721, -0.119]",418.0,-0.120362,-0.739719,-1.706927
"(-0.119, 1.482]",479.0,1.465602,0.535186,-0.116661
"(1.482, 3.084]",57.0,3.083671,1.965297,1.482792


In [79]:
grouping = pd.qcut(x = frame['data1'], q = 10, labels = False)
grouped = frame['data2'].groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,3.083671,0.058590,-2.344311
1,100.0,2.218083,0.051975,-2.202248
2,100.0,2.291170,-0.090110,-2.799182
3,100.0,2.685439,-0.050958,-2.310150
4,100.0,2.158520,-0.002078,-2.405836
5,100.0,2.918493,0.051328,-2.567874
6,100.0,2.202372,-0.240909,-3.322435
7,100.0,2.490692,-0.073362,-2.812656
8,100.0,2.564643,-0.124309,-2.182193


### Filter Missing Values with Group-Specific Values

When cleaning up missing data, in some case you will filter out data observations using `dropna`, but in others you many want to impute(fill in) the NA values using a fixed value or some value derived from data.

`fillna` is the right tool to use.

In [80]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.130940
2         NaN
3    0.423415
4         NaN
5   -0.344154
dtype: float64

In [81]:
s.fillna(s.mean()) # fill missing value by mean

0   -0.350560
1   -1.130940
2   -0.350560
3    0.423415
4   -0.350560
5   -0.344154
dtype: float64

Suppose you need the fill value to vary by group. 

***You need only group the data and use `apply` with function that calls `fillna` on wach data chunk.***

In [108]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 
          'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4

data = pd.Series(data = np.random.rand(8), index = states)

data[["Vermont", 'Nevada', 'Idaho']] = np.nan

data

Ohio          0.516040
New York      0.659189
Vermont            NaN
Florida       0.581309
Oregon        0.316001
Nevada             NaN
California    0.065279
Idaho              NaN
dtype: float64

In [109]:
data.groupby(group_key).mean()

East    0.585512
West    0.190640
dtype: float64

In another case, you might have pre-defined fill values in your code that vary by group.

***Since the groups have a `name` attribute set internally.***

In [110]:
fill_values = {'East': 0.5, 'West': -1}

In [111]:
fill_func = lambda group: group.fillna(fill_values[group.name])

In [112]:
print(data)
print("-"*50)
data.groupby(group_key).apply(fill_func)

Ohio          0.516040
New York      0.659189
Vermont            NaN
Florida       0.581309
Oregon        0.316001
Nevada             NaN
California    0.065279
Idaho              NaN
dtype: float64
--------------------------------------------------


Ohio          0.516040
New York      0.659189
Vermont       0.500000
Florida       0.581309
Oregon        0.316001
Nevada       -1.000000
California    0.065279
Idaho        -1.000000
dtype: float64

### Random Sampling and Permutation

Suppose you wanted to draw a random sample(with or without replacement) from a large dataset for Monte Carlo simulation purpose or some other application.

There are number of ways to perform the `draws`, some are numch more efficient than others.

***One way is to select the first k element of `np.random.permutation(N)`, where N is the size of your complete dataset and k the desired sample size.***

In [113]:
# Hearts, Spades, Clubs, Diamonds
suits = ["H", "S", "C", "D"]
base_name = list(list(range(1, 11)) + [10]*3)*4
card_val = ['A'] + list(range(2, 11)) + ["J", "Q", "K"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in card_val)
    

In [114]:
deck = pd.Series(data = base_name, index = cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [115]:
def draw(deck, n = 5):
    return deck.take(np.random.permutation(len(deck))[:n])

In [116]:
draw(deck)

6S     6
QD    10
7C     7
4D     4
7H     7
dtype: int64

In [117]:
get_suit = lambda card: card[-1] ## last letter is suit

In [118]:
deck.groupby(get_suit).apply(draw, n = 2)

C  8C     8
   5C     5
D  7D     7
   5D     5
H  4H     4
   8H     8
S  4S     4
   JS    10
dtype: int64

***`groupby` 如果輸入的是 function，且未指定 `axis`，則通常都是對 `Series` 或 `DataFrame` 的 `Index`，進行操作 ***

`aggregate`、`transform` and `apply`

`aggregate` 對資料表進行統計與彙整，產出來的會是一個 scale

`transform` 對資料表進行統計與彙整，並利用彙整出來的值，覆寫資料

`apply` 對資料進行自訂函數的操作(活用度較高)

In [119]:
gro = list(map(get_suit, cards))
gro[:3]

['H', 'H', 'H']

In [122]:
deck.groupby(gro).apply(draw, n = 2)

C  QC    10
   JC    10
D  3D     3
   JD    10
H  3H     3
   4H     4
S  JS    10
   8S     8
dtype: int64

### Group Weighted Average and Correlation

Under the split-apply-combine paradigm of `groupby`, operation between columns
in a DataFrame or two Series, such a group weighted average, become a routine affair.

The group weighted average by `category`.

In [121]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.439980,0.925592
1,a,0.529648,0.670881
2,a,0.119156,0.368513
3,a,-0.822555,0.115408
4,b,0.829337,0.201373
5,b,-0.685836,0.987823
6,b,-0.299799,0.529799
7,b,1.606644,0.500390


In [127]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights = g['weights'])
grouped.apply(get_wavg)

category
a    0.342028
b    0.060664
dtype: float64

In [128]:
close_px = pd.read_csv(".\\pydata-book\\examples\\stock_px.csv", 
                       parse_dates=True, index_col=0)
close_px[:3]

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,4.98,7.86,2.87,16.79,4.27,0.51,6.04,328.79,6.12
1990-02-02,5.04,8.00,2.87,16.89,4.37,0.51,6.09,330.92,6.24
1990-02-05,5.07,8.18,2.87,17.32,4.34,0.51,6.05,331.85,6.25


Our task of interest might be compute a DataFrame consising of the yearly correlations of daily retruns(computed from percent change) with SPX.

In [133]:
rets = close_px.pct_change().dropna()
rets[:5]

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-02,0.012048,0.017812,0.000000,0.005956,0.023419,0.0,0.008278,0.006478,0.019608
1990-02-05,0.005952,0.022500,0.000000,0.025459,-0.006865,0.0,-0.006568,0.002810,0.001603
1990-02-06,-0.011834,-0.007335,0.003484,0.013857,-0.004608,0.0,0.016529,-0.006599,-0.003200
1990-02-07,0.005988,-0.043103,0.010417,0.021071,0.013889,0.0,0.003252,0.012407,0.016051
1990-02-08,0.000000,-0.007722,0.003436,-0.003904,0.018265,0.0,0.008104,-0.002367,0.003160


`corrwith` : Compute pairwise correlation between rows or columns of two DataFrame objects.

***`corrwith` is behaving similarly to `add`, `sub`, `mul`, `div` in that it expects to find a DataFrame or a Series being passed in other despite the documentation saying just DataFrame.***

`corr`: between rows or columns of two DataFrame objects.

***Use `corr` to compare numerical columns within the same dataframe. Non-numerical columns will automatically be skipped.***

***You can compare columns of df1 & df2 with `corrwith`. Note that only columns with the same names are compared.***

In [140]:
np.random.seed(123)
df1 = pd.DataFrame(np.random.randn(3,2), columns=list('ab') )
df2 = pd.DataFrame(np.random.randn(3,2), columns=list('ac') )

In [142]:
df1.corrwith(other = df2)

a    0.993085
b         NaN
c         NaN
dtype: float64

### Additional Optionas

If you want to ignore the column names just compare the first row of df1 to the first row of df2, then you could rename the columns of df2 to match the columns of df1 like this:

***Note that df1 and df2 need to have the same number of columns in that case.***

In [144]:
df1.corrwith(df2.set_axis( df1.columns, axis='columns', inplace=False))

a    0.993085
b    0.969220
dtype: float64

In [160]:
df1['a'].corr(df2['a'])

0.9930848669780297

In [134]:
spx_corr = lambda x : x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year) # x represent the each group
by_year.apply(spx_corr)[:5]

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990,0.595024,0.545067,0.752187,0.738361,0.801145,0.586691,0.783168,1.0,0.517586
1991,0.453574,0.365315,0.759607,0.557046,0.646401,0.524225,0.641775,1.0,0.569335
1992,0.398180,0.498732,0.632685,0.262232,0.515740,0.492345,0.473871,1.0,0.318408
1993,0.259069,0.238578,0.447257,0.211269,0.451503,0.425377,0.385089,1.0,0.318952
1994,0.428549,0.268420,0.572996,0.385162,0.372962,0.436585,0.450516,1.0,0.395078


In [146]:
# Annual correlation of Apple with Microsoft

by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))[-3:]

2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Group - wise Linear Regression

You can use `groupby` to perform more complex group-wise statistical analysis, as long as the function returns a pandas object or scalar value.

You can also define the following `regress` function(using the statsmodels econometrics libary) which excute an ordinary least squares(OLS) regression on each chunk of data. 

In [165]:
from pandas.core import datetools
import statsmodels.api as sm

In [166]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [167]:
by_year.apply(regress, 'AAPL', ['SPX'])[:5]

,SPX,intercept
1990,1.512772,0.001395
1991,1.187351,0.000396
1992,1.832427,0.000164
1993,1.390470,-0.002657
1994,1.190277,0.001617


### Pivot Tables and Cross-Tabulation

A `pivot table` is a data summarization tool frequently found in spreadsheet programs and other data analysis software.

I aggregates a table of data by on or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along columns.

***Pivot tables in python with pandas are made possible using the `groupby` facility described in this chapter combined with reshape operations utilizing hierarchical indexing.***

***DataFrame has a `pivot_table` method, and additionally there is a top-level `pandas.pivot_table` function.***

In addition to providing a convenience interface to `groupby`, `pivot_table` also can add partial totals, also known as `margins`.

In [169]:
tips.pivot_table(index = ['time', 'smoker'])

size       tip   tip_pct  total_bill
time   smoker                                          
Dinner No      2.735849  3.126887  0.158653   20.095660
       Yes     2.471429  3.066000  0.160828   21.859429
Lunch  No      2.511111  2.673778  0.160920   17.050889
       Yes     2.217391  2.834348  0.170404   17.399130

In [170]:
tips.pivot_table(index = ['smoker', 'time'])

size       tip   tip_pct  total_bill
smoker time                                            
No     Dinner  2.735849  3.126887  0.158653   20.095660
       Lunch   2.511111  2.673778  0.160920   17.050889
Yes    Dinner  2.471429  3.066000  0.160828   21.859429
       Lunch   2.217391  2.834348  0.170404   17.399130

This could have been easily produced using `groupby`. Now suppose we want to aggregate only `tip_pct` and `size`, and additionally group by `day`.

I will put `smoker` in the table columns and `day` in the rows.

***This table could be augmented to include partial totals by passing `margin = True`.***

This has the effect of adding all row and column labels, with corresponding values being the group statistics for all the data withub a single tier.

The **All** values are means without taking into account smoker v.s non-smoker(the **All** columns) or any of the two levels of grouping on the rows.

In [181]:
tips.pivot_table(values = ['tip_pct', 'size'], index = ['day'],
                 columns = ['smoker'], margins = True)

size                       tip_pct                    
smoker        No       Yes       All        No       Yes       All
day                                                               
Fri     2.250000  2.066667  2.105263  0.151650  0.174783  0.169913
Sat     2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
Sun     2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
Thur    2.488889  2.352941  2.451613  0.160298  0.163863  0.161276
All     2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

To use a different aggregation function, pass it tot `aggfun`.

`count` or `len` will give you a cross-tabulation(count or frequency) of **group sizes**.

In [200]:
tips.pivot_table(values = ['tip_pct', 'size'], index = ['time', 'smoker'], 
                 columns = ['day'], aggfunc = len, margins = True)

size                        tip_pct                         
day             Fri   Sat   Sun  Thur  All     Fri   Sat   Sun  Thur    All
time   smoker                                                              
Dinner No       3.0  45.0  57.0   1.0  106     3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70     9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45     1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23     6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244    19.0  87.0  76.0  62.0  244.0

If some combinations are empty(or otherwise NA), you may wish to pass `fill_value`.

In [199]:
tips.pivot_table(values = 'size', index = ['time', 'smoker'],
                 columns = ['day'], aggfunc = sum, fill_value = 0)

day            Fri  Sat  Sun  Thur
time   smoker                     
Dinner No        6  115  167     2
       Yes      20  104   49     0
Lunch  No        3    0    0   110
       Yes      11    0    0    40